# Transformerの学習・推論・判定根拠の可視化
TransformerモデルとIMDbのデータローダを使用してクラス分類（２値）を学習させる。<br>
テストデータで推論し、判断根拠となるAttentionを可視化する

## Library

In [2]:
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim

import torchtext

OSError: dlopen(/Users/eri/opt/anaconda3/lib/python3.7/site-packages/torchtext/_torchtext.so, 6): Library not loaded: @rpath/libtorch_cpu.dylib
  Referenced from: /Users/eri/opt/anaconda3/lib/python3.7/site-packages/torchtext/_torchtext.so
  Reason: image not found

In [ ]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

## DatasetとDataLoader

In [3]:
from utils.dataloader import get_IMDb_DataLoaders_and_TEXT

# 読み込み
train_dl, val_dl, test_dl, TEXT = get_IMDb_DataLoaders_and_TEXT(max_length=256, batch_size=64)
dataloaders_dict = {'train': train_dl, 'val': val_dl}

ModuleNotFoundError: No module named 'spacy'

## ネットワークモデルの作成

In [4]:
from utils.transformer import TransformerClassification

# モデル構築
net = TransformerClassification(
    text_embedding_vectors=TEXT.vocab.vectors, d_model=300, max_seq_len=256, output_dim=2)

# ネットワーク初期化
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
            
# 訓練モードに設定
net.train()

# TransformerBlockモジュールを初期化実行
net.net3_1.apply(weights_init)
net.net3_2.apply(weights_init)

print('ネットワーク設定完了')

ModuleNotFoundError: No module named 'spacy'

## 損失関数と最適化手法の定義

In [5]:
# 分類なのでcrossentropy
criterion = nn.CrossEntropyLoss()

# adamを使って最適化
learning_rare = 2e-5
optimizer =optim.Adam(net.parameters(), lr=learning_rare)

NameError: name 'net' is not defined

## 学習・検証の実施

In [ ]:
# 学習・検証を実行
num_epochs = 10
net.trained = train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)


## テストデータでの正解率を求める

In [ ]:
# device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()
net_trained.to(device)

epoch_corrects = 0

for batch in (test_dl):
    # batchはTextとLabelの辞書オブジェクト
    inputs = batch.Text[0].to(device)
    labels = batch.Label.to(device)
    
    with torch.set_grad_enable(False):
        
        # mask作成
        input_pad = 1
        input_mask = (inputs != input_pad)　　# padのところはFalseになる
        
        # transformerに入力
        outputs, _, _ = net_trained(inputs, input_mask)
        _, preds = torch.max(outputs, 1)  # ラベルと予測
        
        # 結果の計算
        # 正解数の合計を更新
        epoch_corrects += torch.sum(preds==labels.data)
        
# 正解率
epoch_acc = epoch_corrects.double() / len(test_dl.dataset)

print('テストデータ{}個での正解率:{:.4f}'.format(len(test_dl.dataset), epoch_acc))

## Attentionの可視化で判定根拠を探る

In [7]:
# htmlを作成する関数
def hightlight(word, attn):
    """
    Attentionの値が大きいと文字の背景が濃い赤になるhtmlを出力させる関数
    """
    
    html_color = '#%02X%02X%02X' % (
        255, int(255*(1-attn)), int(255*(1-attn)))
    return '<span style="background-color: {}"> {}</span>'.format(html_color, word)

def mk_html(index, batch, preds, normalized_weights_1, normalized_weights_2,TEXT):
    """
    htmlデータを作成する
    """
    
    # indexの結果を抽出
    sentence = batch.Text[0][index]  # 文章
    label = batch.Label[index]  # ラベル
    pred = preds[index]  # 予測
    
    # indexのAttentinoを抽出と規格化
    attens1 = normalized_weights_1[index, 0, :]  # 0番目の<cls>のAttention
    attens1 /= attens1.max()
    
    attens2 = normalized_weights_2[index, 0, :]
    attens2 /= attens2.max()
    
    # ラベルと予測結果を文字に置き換え
    if label == 0:
        label_str = 'Negative'
    else:
        label_str = 'Positive'
        
    if pred == 0:
        pred_str = 'Negative'
    else:
        pred_str = 'Positive'
        
    # 表示用のHTMLを作成する
    html = '正解ラベル：{}<br>推論ラベル : {}<br><br>'.format(label_str, pred_str)
    
    # 一段目のAttention
    html += '[TransformerBlockの1段目のAttentionを可視化]<br>'
    for word, attn in zip(sentence, attens1):
        html += hightlight(TEXT.vocab.itos[word], attn)
    html += '<br><br>'
    
    # 二段目のAttention
    html += '[TransformerBlockの二段目のAttentionを可視化]<br>'
    for word, attn in zip(sentence, attens2):
        html += hightlight(TEXT.vocab.itos[word], attn)
    html += '<br><br>'
    
    return html

In [10]:
from IPython.display import HTML

# Transformerで処理
# ミニバッチの用意
batch = next(iter(test_dl))

inputs = batch.Text[0].to(device)
labels = batch.Label.to(device)

input_pad = 1
input_mask = (inputs != input_pad)  # padのところはFalseになる

# Transformerに入力
outputs, normalized_weights_1, normalized_weights_2 = net_trained(
    inputs, input_mask)
_, preds = torch.max(outputs, 1)  # ラベルの予測

index = 3  # 出力させたいデータ　　3文目
# HTMLを作成
html_output = mk_html(index, batch, preds, normalized_weights_1, normalized_weights_2, TEXT)
# HTML形式で出力
HTML(html_output)

NameError: name 'test_dl' is not defined